In [ ]:
from djitellopy import tello
import cv2
import numpy as np
import time

In [ ]:
network = cv2.dnn.readNetFromCaffe("./models/deploy.prototxt", "./models/res10_300x300_ssd_iter_140000.caffemodel")

network.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
network.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

def detectFaces(net, frame, pixel ,conf_threshold=0.7):
    areas=[]
    centers=[]

    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], False, False,)

    net.setInput(blob)
    detections = net.forward()
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight / 150)), 8,)
            
            top=x1
            right=y1
            bottom=x2-x1
            left=y2-y1

            areas.append(bottom*left)

            cx=(x1+x2)//2
            cy=(y1+y2)//2
            centers.append([cx,cy])
            
            cv2.circle(frame,(cx,cy),5,(0,255,0),cv2.FILLED)
            if(pixel):
                #  blurry rectangle to the detected face
                face = frame[right:right+left, top:top+bottom]
                #desenfocar cara
                face = cv2.GaussianBlur(face,(23, 23), 30)
                frame[right:right+face.shape[0], top:top+face.shape[1]] = face

    return frame, bboxes, areas, centers

In [ ]:
def faceTracking(drone,area,center):
    minX=130
    minY=120
    minArea=8000

    maxX=170
    maxY=180
    maxArea=15000
    

    #velocity
    leftRight=0
    forwardBackward=0
    upDown=0
    yaw=0

    if(center[0]>maxX):
        #drone.rotate_clockwise(10)
        yaw=25
    elif(center[0]<minX):
        #drone.rotate_counter_clockwise(10)
        yaw=-25
    
    if(center[1]>maxY):
        #drone.move("down",20)
        upDown=-25
    elif(center[1]<minY):
        #drone.move("up",20)
        upDown=25
        
    if(area>maxArea):
        #drone.move("back",20)
        forwardBackward=-20
    elif(area<minArea):
        #drone.move("forward",20)
        forwardBackward=20
        
    drone.send_rc_control(leftRight,forwardBackward,upDown,yaw)

    

In [ ]:
def move(direction,vel,drone):
    leftRight=0
    forwardBackward=0
    upDown=0
    yaw=0
    if(direction=="up"):
        upDown=vel
    elif(direction=="down"):
        upDown=-vel
    elif(direction=="forward"):
        forwardBackward=vel
    elif(direction=="back"):
        forwardBackward=-vel
    elif(direction=="left"):
        yaw=-vel
    elif(direction=="right"):
        yaw=vel
    
    drone.send_rc_control(leftRight,forwardBackward,upDown,yaw)

In [ ]:
me=tello.Tello()
me.connect()
print(me.get_battery())

In [ ]:

me.streamon()


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (300,300))

In [ ]:
img = np.random.randint(222, size=(100, 100,3))
detectionEnabled = False
pixel=False
facetrack=False
is_flying=False
while True:
    frameRead=me.get_frame_read()

    time.sleep(0.05)
    
    if frameRead :
        frame= cv2.resize(frameRead.frame,(300,300))
        
        if(detectionEnabled == True):
            outOpencvDnn, bboxes, areas, centers = detectFaces(network, frame, pixel,0.85)
            #cv2.imshow("Output",frame)

            #if(bboxes):
                #print("area: ")
                #print(areas[0])
                #print("centro: ")
                #print(centers[0])
            if(facetrack and bboxes):
                faceTracking(me,areas[0],centers[0])
            elif(facetrack and (not bboxes)):
                me.send_rc_control(0,0,0,0)
                
        #else:
        cv2.imshow("Output",frame)
        out.write(frame)
        
    # key controller
    
        #u up       #w forward
        #d down     #s back
        #a left     #d right
    
        #f enable or disable face detection
        #p enable or disable face pixeling
        #
    #
    key = cv2.waitKey(1) & 0xFF    
    if key == ord("f"):
        detectionEnabled = not detectionEnabled
    
    elif key == ord("p"):
        pixel=not pixel
        if((not detectionEnabled) and (facetrack)): 
            detectionEnabled = not detectionEnabled
        if((not pixel) and (detectionEnabled)):
            detectionEnabled = not detectionEnabled

    elif key==ord("k"):
        facetrack=not facetrack
        if((not detectionEnabled) and (facetrack)): 
            detectionEnabled = not detectionEnabled
        if((not facetrack) and (detectionEnabled)):
            detectionEnabled = not detectionEnabled

    
    elif key == ord("t"):
        if( not is_flying):
            me.takeoff()
            is_flying= True
    elif key == ord("l"):
        if(is_flying):
            me.land()
            is_flying=False
            
    elif(not facetrack):
        
        if key == ord('u'):
            move('up',20,me)
        elif key == ord('j'):
            move('down',20,me) 
        elif key == ord("w"):
            move('forward',20,me)
        elif key == ord('a'):
            move('left',20,me)
        elif key == ord('s'):
            move('back',20,me)
        elif key == ord("d"):
            move('right',20,me)

    if key == ord("q"):
        break
out.release()
if(me.is_flying):
    me.land()
me.streamoff()
cv2.destroyAllWindows()